In [1]:
import os

import numpy as np
from qonnx.core.modelwrapper import ModelWrapper

import finn.core.onnx_exec as oxe

from qonnx.custom_op.registry import getCustomOp

from finn.util.basic import pynq_part_map
import pandas as pd

from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.create_stitched_ip import CreateStitchedIP

from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim

from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.analysis.fpgadataflow.dataflow_performance import dataflow_performance
from finn.util.basic import (
    get_rtlsim_trace_depth,
    pyverilate_get_liveness_threshold_cycles,
)

# Setup Folders and Model Filenames

In [2]:
experiment_folder = './experiments/035_pynq-z1__workspace__w4W2a4__full_build/output_full_build'
model_folder = experiment_folder + '/intermediate_models'
stitched_ip = model_folder + '/step_create_stitched_ip.onnx'
parent_model = model_folder + '/dataflow_parent.onnx'

stitched_rtlsim_filename = "./models/stitched_rtlsim.onnx"

# Function to prepare model for RTLSim: FIFOs impl_style from "vivado" to "rtl"

In [3]:
def prepare_for_stitched_ip_rtlsim(verify_model, fpga_part, target_clk_ns):
    need_restitch = False
    # switch impl_style=vivado components to rtl
    # StreamingFIFO must have impl_style=rtl
    for fifo_layer in verify_model.get_nodes_by_op_type("StreamingFIFO_rtl"):
        inst = getCustomOp(fifo_layer)
        if inst.get_nodeattr("impl_style") != "rtl":
            inst.set_nodeattr("impl_style", "rtl")
            inst.set_nodeattr("code_gen_dir_ipgen", "")
            inst.set_nodeattr("ipgen_path", "")
            need_restitch = True
    # if we've made alterations to the model, need to do some re-prep
    if need_restitch:
        print("Need to regen/re-stitch some IP for STITCHED_IP_RTLSIM")
        verify_model = verify_model.transform(
            PrepareIP(fpga_part, target_clk_ns)
        )
        verify_model = verify_model.transform(HLSSynthIP())
        verify_model = verify_model.transform(
            CreateStitchedIP(
                fpga_part,
                target_clk_ns,
                vitis=False,
            )
        )
  
    # set top-level prop for stitched-ip rtlsim and launch
    verify_model.set_metadata_prop("exec_mode", "rtlsim")
    # TODO make configurable
    # verify_model.set_metadata_prop("rtlsim_trace", "trace.vcd")
    return verify_model

# PYNQ Boards Available

In [4]:
fpga_df = pd.DataFrame(pynq_part_map.items(), columns=['Board', 'FPGA Part'])
fpga_df

,Board,FPGA Part
0,Ultra96,xczu3eg-sbva484-1-e
1,Ultra96-V2,xczu3eg-sbva484-1-i
2,Pynq-Z1,xc7z020clg400-1
3,Pynq-Z2,xc7z020clg400-1
4,ZCU102,xczu9eg-ffvb1156-2-e
5,ZCU104,xczu7ev-ffvc1156-2-e
6,ZCU111,xczu28dr-ffvg1517-2-e
7,RFSoC2x2,xczu28dr-ffvg1517-2-e
8,RFSoC4x2,xczu48dr-ffvg1517-2-e
9,KV260_SOM,xck26-sfvc784-2LV-c


In [5]:
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
print(fpga_part)

xc7z020clg400-1


In [6]:
synth_clk_period_ns = 10.0

# Create New Stitched IP with impl_style = "rtl"

Only needed the first time, to create the new stitched IP. Once it is created, this step can be skipped

In [7]:
# child_model = ModelWrapper(stitched_ip)

# child_model = prepare_for_stitched_ip_rtlsim(child_model, fpga_part, synth_clk_period_ns)

# child_model = child_model.transform(PrepareRTLSim())
# #child_model.set_metadata_prop("exec_mode","rtlsim")
# child_model.save(stitched_rtlsim_filename)

# Load Parent Model and link Partition (Child) to Stitched IP for RTLSim

In [8]:
# parent model
model_for_rtlsim = ModelWrapper(parent_model)
# reference child model
sdp_node = getCustomOp(model_for_rtlsim.graph.node[1])
sdp_node.set_nodeattr("model", stitched_rtlsim_filename)

# LIVENESS_THRESHOLD: adjust it to model Max Cycles -> Critical Path

In [9]:
stiched_rtlsim_model = ModelWrapper(stitched_rtlsim_filename)
stiched_rtlsim_model = stiched_rtlsim_model.transform(AnnotateCycles())
estimate_network_performance = stiched_rtlsim_model.analysis(dataflow_performance)
prev_liveness = pyverilate_get_liveness_threshold_cycles()

In [10]:
print(f'Previous LIVENESS_THRESHOLD = {prev_liveness}')
print(f'Estimated performance of Stuched_RTLSim\n{estimate_network_performance}')

Previous LIVENESS_THRESHOLD = 10000
Estimated performance of Stuched_RTLSim
{'critical_path_cycles': 180502470, 'max_cycles': 43352064, 'max_cycles_node_name': 'MVAU_hls_0'}


In [11]:
os.environ["LIVENESS_THRESHOLD"] = str(
    int(estimate_network_performance["critical_path_cycles"])
)
#os.environ["LIVENESS_THRESHOLD"] = str(prev_liveness)

In [12]:
print(os.environ["LIVENESS_THRESHOLD"])

180502470


# Eval with DFire MINI

In [13]:
DS_FOLDER = '../../datasets/dfire_mini/'
DS_FOLDER += 'test/'
# DS_FOLDER += 'train/'
DS_IMGS = DS_FOLDER + 'images/'
DS_LABELS = DS_FOLDER + 'labels/'

In [14]:
IMGS_LIST = sorted([DS_IMGS + img_file for img_file in os.listdir(DS_IMGS)])

In [15]:
import modules.my_metrics as metrics

In [16]:
metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = model_for_rtlsim,
    bipolar=True,
    divide_255=False) # Preprocess included

Number of test samples: 30



Testing:   0%|                                                                                                                                                        | 0/30 [00:00<?, ?it/s]%Warning-MODDUP: /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:48499:8: Duplicate declaration of module: 'swg_controller'
48499 | module swg_controller
      |        ^~~~~~~~~~~~~~
                 /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:22380:8: ... Location of original declaration
22380 | module swg_controller
      |        ^~~~~~~~~~~~~~
                 ... For warning description see https://verilator.org/warn/MODDUP?v=4.224
                 ... Use "/* verilator lint_off MODDUP */" and lint_on around source to disable this message.
%Warning-MODDUP: /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:48614:8: Duplicate declaration of module: 'swg_cyclic_buffer_addressable'
48614 | module swg_cyclic_buffer_addre

make: Entering directory '/home/gmoreno/workspace/pyverilator_ipstitched_i37s0wex'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /home/gmoreno/workspace/pyverilator_ipstitched_i37s0wex/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/lo

Testing:   3%|████▋                                                                                                                                       | 1/30 [17:34<8:29:54, 1054.99s/it]

Image file: ../../datasets/dfire_mini/test/images/AoF06872.jpg: 
	label [0. 0.]
	pred [0. 0.]


...

Image file: ../../datasets/dfire_mini/test/images/WEB11514.jpg: 
	label [1. 1.]
	pred [1. 1.]


%Warning-MODDUP: /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:48499:8: Duplicate declaration of module: 'swg_controller'
48499 | module swg_controller
      |        ^~~~~~~~~~~~~~
                 /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:22380:8: ... Location of original declaration
22380 | module swg_controller
      |        ^~~~~~~~~~~~~~
                 ... For warning description see https://verilator.org/warn/MODDUP?v=4.224
                 ... Use "/* verilator lint_off MODDUP */" and lint_on around source to disable this message.
%Warning-MODDUP: /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:48614:8: Duplicate declaration of module: 'swg_cyclic_buffer_addressable'
48614 | module swg_cyclic_buffer_addressable #(
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                 /home/gmoreno/workspace/vivado_stitch_proj_w2a9_ghn/finn_design_wrapper.v:22495:8: ... Location of original declarati

make: Entering directory '/home/gmoreno/workspace/pyverilator_ipstitched_llatbzl7'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o pyverilator_wrapper.o /home/gmoreno/workspace/pyverilator_ipstitched_llatbzl7/pyverilator_wrapper.cpp
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow     -fPIC --std=c++11  -std=gnu++17 -Os -c -o verilated.o /usr/lo

Testing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [8:40:23<00:00, 1040.80s/it]

Image file: ../../datasets/dfire_mini/test/images/WEB11588.jpg: 
	label [1. 0.]
	pred [1. 0.]
Smoke -> TP: 20.0, FP: 1.0, TN: 7.0, FN: 2.0
Fire  -> TP: 11.0, FP: 0.0, TN: 19.0, FN: 0.0


# Print Results

In [19]:
metrics_df = pd.DataFrame(metrics)
metrics_df

,Smoke,Fire,Mean
Accuracy,0.9000,1.0,0.9500
Precision,0.9524,1.0,0.9762
Recall,0.9091,1.0,0.9546
F1,0.9302,1.0,0.9651
